# **<font color='crimson'>«Список из пяти последних совершенных (выполненных) операций клиента»</font>**

---

**Выполнил**: Юмаев Егор

---

## <font color='green'>**ВВЕДЕНИЕ. ПОСТАНОВКА ЗАДАЧИ**</font>

---

Имеются сведения о совершенных клиентом в разное время операциях. По каждой операции собрана следующая информация:

- **date** - информация о дате совершения операции
- **state** - статус перевода (EXECUTED - выполнена, CANCELED - отменена)
- **operationAmount** - сумма операции и валюта
- **description** - описание типа перевода
- **from** - откуда
- **to** - куда


**Задача**: сформировать список из пяти последних совершенных (выполненных) операций клиента в формате:

<дата перевода> <описание перевода><откуда> -> <куда><сумма перевода> <валюта>

Описание каждой операции должно быть представлено на отдельной строке.

Описание каждой операции должно соответствовать следующим **условиям**:

- дата перевода должна быть в формате ДД.ММ.ГГГГ

- сверху списка должны быть самые последние операции (по дате)

- номер карты должен маскироваться и не отображаться целиком, в формате XXXX XX** **** XXXX (видны первые 6 цифр и последние 4, разбито по блокам по 4 цифры, разделенных пробелом)

- номер счета должен маскироваться и не отображаться целиком, в формате **XXXX (видны только последние 4 цифры номера счета)


## <font color='green'>**ЗАГРУЗКА ДАННЫХ**</font>

---

In [ ]:
# импортируем необходимую библиотеку
import pandas as pd

In [ ]:
# подгружаем данные
! gdown 1oE85r4qJdE8jZHyp0_gZAiDb4Vb9m0cu

Downloading...
From: https://drive.google.com/uc?id=1oE85r4qJdE8jZHyp0_gZAiDb4Vb9m0cu
To: /content/operations.json
100% 37.5k/37.5k [00:00<00:00, 62.9MB/s]


In [ ]:
# сохраним данные в датафрейм
df = pd.read_json('/content/operations.json')

In [ ]:
# проверим размеры таблицы
df.shape

(101, 7)

In [ ]:
# выведем первые пять строк таблицы
df.head()

,id,state,date,operationAmount,description,from,to
0,441945886.0,EXECUTED,2019-08-26 10:50:58.294041,"{'amount': '31957.58', 'currency': {'name': 'р...",Перевод организации,Maestro 1596837868705199,Счет 64686473678894779589
1,41428829.0,EXECUTED,2019-07-03 18:35:29.512364,"{'amount': '8221.37', 'currency': {'name': 'US...",Перевод организации,MasterCard 7158300734726758,Счет 35383033474447895560
2,939719570.0,EXECUTED,2018-06-30 02:08:58.425572,"{'amount': '9824.07', 'currency': {'name': 'US...",Перевод организации,Счет 75106830613657916952,Счет 11776614605963066702
3,587085106.0,EXECUTED,2018-03-23 10:45:06.972075,"{'amount': '48223.05', 'currency': {'name': 'р...",Открытие вклада,NaN,Счет 41421565395219882431
4,142264268.0,EXECUTED,2019-04-04 23:20:05.206878,"{'amount': '79114.93', 'currency': {'name': 'U...",Перевод со счета на счет,Счет 19708645243227258542,Счет 75651667383060284188


Данные корректно загружены и готовы для проведения анализа.

## <font color='green'>**ПРЕДОБРАБОТКА ДАННЫХ**</font>

---

In [ ]:
# приведем название признака operationAmount к snake_case
df = df.rename(columns={'operationAmount': 'operation_amount'})

In [ ]:
# проверим, какие есть статусы в признаке status
df['state'].value_counts(dropna = False)

state
EXECUTED    85
CANCELED    15
NaN          1
Name: count, dtype: int64

In [ ]:
# проверим, сколько уникальных id операций представлено в таблице
df['id'].nunique()

100

In [ ]:
# проверим, есть ли дубликаты id в таблице
df['id'].duplicated().sum()

0

In [ ]:
# сортируем операции по дате совершения
df = df.sort_values('date', ascending = False)
df.head()

,id,state,date,operation_amount,description,from,to
11,863064926.0,EXECUTED,2019-12-08 22:46:21.935582,"{'amount': '41096.24', 'currency': {'name': 'U...",Открытие вклада,NaN,Счет 90424923579946435907
70,114832369.0,EXECUTED,2019-12-07 06:17:14.634890,"{'amount': '48150.39', 'currency': {'name': 'U...",Перевод организации,Visa Classic 2842878893689012,Счет 35158586384610753655
56,560813069.0,CANCELED,2019-12-03 04:27:03.427014,"{'amount': '17628.50', 'currency': {'name': 'U...",Перевод с карты на карту,MasterCard 1796816785869527,Visa Classic 7699855375169288
40,154927927.0,EXECUTED,2019-11-19 09:22:25.899614,"{'amount': '30153.72', 'currency': {'name': 'р...",Перевод организации,Maestro 7810846596785568,Счет 43241152692663622869
72,482520625.0,EXECUTED,2019-11-13 17:38:04.800051,"{'amount': '62814.53', 'currency': {'name': 'р...",Перевод со счета на счет,Счет 38611439522855669794,Счет 46765464282437878125


In [ ]:
# отфильтруем только те строки таблицы, в признаке 'status' которых
# указано 'EXECUTED'
df = df[df['state'] == 'EXECUTED']

In [ ]:
# проверим, какие есть статусы в признаке status
df['state'].value_counts(dropna = False)

state
EXECUTED    85
Name: count, dtype: int64

Удалены все строки, в которых был указан статус **'CANCELED'** и одна строка, в которой статус был не указан. По умолчанию считаем, что запись с **NAN** в признаке **'state'** означает, что операция не была выполнена.

## <font color='green'>**ОБРАБОТКА СВЕДЕНИЙ ОБ ОПЕРАЦИЯХ КЛИЕНТА**</font>

---

Поскольку согласно техзадания требуется вывести сведения о пяти самых последних операциях, создадим копию таблицы, а работу продолжим с пятью строками с самыми последними операциями.

In [ ]:
# создаем копию таблицы
df_copy = df.copy()

In [ ]:
# сохраним первые пять строк таблицы, необходимые для отчета
df = df[0:5]

In [ ]:
# выведем таблицу df
df

,id,state,date,operation_amount,description,from,to
11,863064926.0,EXECUTED,2019-12-08 22:46:21.935582,"{'amount': '41096.24', 'currency': {'name': 'U...",Открытие вклада,NaN,Счет 90424923579946435907
70,114832369.0,EXECUTED,2019-12-07 06:17:14.634890,"{'amount': '48150.39', 'currency': {'name': 'U...",Перевод организации,Visa Classic 2842878893689012,Счет 35158586384610753655
40,154927927.0,EXECUTED,2019-11-19 09:22:25.899614,"{'amount': '30153.72', 'currency': {'name': 'р...",Перевод организации,Maestro 7810846596785568,Счет 43241152692663622869
72,482520625.0,EXECUTED,2019-11-13 17:38:04.800051,"{'amount': '62814.53', 'currency': {'name': 'р...",Перевод со счета на счет,Счет 38611439522855669794,Счет 46765464282437878125
38,801684332.0,EXECUTED,2019-11-05 12:04:13.781725,"{'amount': '21344.35', 'currency': {'name': 'р...",Открытие вклада,NaN,Счет 77613226829885488381


In [ ]:
# создадим признак с датой, отформатированной по требованию техзадания
df['date_short'] = df.loc[:, 'date'].dt.strftime('%d.%m.%Y')

In [ ]:
# проверим, что дата в признаке date_short отображается корректно
df

,id,state,date,operation_amount,description,from,to,date_short
11,863064926.0,EXECUTED,2019-12-08 22:46:21.935582,"{'amount': '41096.24', 'currency': {'name': 'U...",Открытие вклада,NaN,Счет 90424923579946435907,08.12.2019
70,114832369.0,EXECUTED,2019-12-07 06:17:14.634890,"{'amount': '48150.39', 'currency': {'name': 'U...",Перевод организации,Visa Classic 2842878893689012,Счет 35158586384610753655,07.12.2019
40,154927927.0,EXECUTED,2019-11-19 09:22:25.899614,"{'amount': '30153.72', 'currency': {'name': 'р...",Перевод организации,Maestro 7810846596785568,Счет 43241152692663622869,19.11.2019
72,482520625.0,EXECUTED,2019-11-13 17:38:04.800051,"{'amount': '62814.53', 'currency': {'name': 'р...",Перевод со счета на счет,Счет 38611439522855669794,Счет 46765464282437878125,13.11.2019
38,801684332.0,EXECUTED,2019-11-05 12:04:13.781725,"{'amount': '21344.35', 'currency': {'name': 'р...",Открытие вклада,NaN,Счет 77613226829885488381,05.11.2019


In [ ]:
# признаки id, state и date далее не требуются, удалим их
df = df.drop(['id', 'state', 'date'], axis = 1)

In [ ]:
# проверим корректность удаления признаков
df

,operation_amount,description,from,to,date_short
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Открытие вклада,NaN,Счет 90424923579946435907,08.12.2019
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Перевод организации,Visa Classic 2842878893689012,Счет 35158586384610753655,07.12.2019
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Перевод организации,Maestro 7810846596785568,Счет 43241152692663622869,19.11.2019
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Перевод со счета на счет,Счет 38611439522855669794,Счет 46765464282437878125,13.11.2019
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Открытие вклада,NaN,Счет 77613226829885488381,05.11.2019


In [ ]:
# проверим типы данных признаков
df.dtypes

operation_amount    object
description         object
from                object
to                  object
date_short          object
dtype: object

In [ ]:
# заполним пропуски в признаке 'from'
df['from'] = df['from'].fillna('Источник не указан')

In [ ]:
# проверим заполнение пропусков
df

,operation_amount,description,from,to,date_short
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Открытие вклада,Источник не указан,Счет 90424923579946435907,08.12.2019
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Перевод организации,Visa Classic 2842878893689012,Счет 35158586384610753655,07.12.2019
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Перевод организации,Maestro 7810846596785568,Счет 43241152692663622869,19.11.2019
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Перевод со счета на счет,Счет 38611439522855669794,Счет 46765464282437878125,13.11.2019
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Открытие вклада,Источник не указан,Счет 77613226829885488381,05.11.2019


Начнем составлять описание операции клиента в заданном формате.

In [ ]:
# объединим сведения о дате и описании операции
df['result'] = df['date_short'] + ' ' + df['description']

In [ ]:
# удалим признаки 'date_short' и 'description', далее они не требуются
df = df.drop(['date_short', 'description'], axis = 1)

In [ ]:
# проверим произошедшие изменения
df

,operation_amount,from,to,result
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Источник не указан,Счет 90424923579946435907,08.12.2019 Открытие вклада
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Visa Classic 2842878893689012,Счет 35158586384610753655,07.12.2019 Перевод организации
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Maestro 7810846596785568,Счет 43241152692663622869,19.11.2019 Перевод организации
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Счет 38611439522855669794,Счет 46765464282437878125,13.11.2019 Перевод со счета на счет
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Источник не указан,Счет 77613226829885488381,05.11.2019 Открытие вклада


Обработаем признак **'from'**: номера банковских карт и номера счетов должны маскироваться в соответствии с требованиями техзадания.

In [ ]:
# подготовим функцию, маскирующуюу номер банковской карты
def mask_bank_card(num: str) -> str:
    '''
    Маскировка номера банковской карты:
    - из 16-ти цифр номера должны быть видны
    первые шесть и последние четыре, остальные цифры
    заменяются знаком *
    - номер карты разбивается по блокам по четыре цифры,
    разделенных пробелом

    Params:
    :num: номер банковской карты из 16-ти цифр в формате str

    Return:
    :num: преобразованный номер банковской карты

    Example:
    >>>new_num = mask_bank_card(num)
    '''
    # между первыми 6-ю символами и последними 4-мя
    # добавляем знак маскировки цифры *
    temp = num[:6] + '*' * 6 + num[-4:]
    # создаем пустой список, в который последовательно
    # добавляем нарезки в формате str по четыре символа
    res = []
    while len(temp) > 4:
        res.append(temp[:4])
        temp = temp[4:]
    res.append(temp)
    return ' '.join(res)

In [ ]:
# проверим, что функция маскировки банковской карты корректно отрабатывает
num = '2842878893689012'
print(mask_bank_card(num))

2842 87** **** 9012


In [ ]:
# подготовим функцию, маскирующуюу номер счета
def mask_bank_account(num: str) -> str:
    '''
    Маскировка номера счета:
    - из 20-ти цифр номера должны быть видны
    только последние четыре, перед которыми
    добавляется два знака маскировки: **

    Params:
    :num: номер счета из 20-ти цифр в формате str

    Return:
    :num: преобразованный номер счета

    Example:
    >>>new_num = mask_bank_account(num)
    '''
    return '**' + num[-4:]

In [ ]:
# проверим, что функция маскировки номера счета корректно отрабатывает
num = '38611439522855669794'
print(mask_bank_account(num))

**9794


In [ ]:
# подготовим функцию, обрабатывающую значения признака from
def process_number(row: str) -> str:
    '''
    В полученной строке функция маскирует номер банковской карты
    или номер счета и возвращает строку с замаскированным номером.
    Если строка, поданная на вход, не содержит номер, тогда
    функция возвращает полученную на вход строку

    Params:
    :row: строка с информацией о банковской карте и ее номере
    или о счете и его номере

    Return:
    :row: преобразованная строка

    Example:
    >>>new_row = process_number(row)
    '''
    temp = row.split()
    # если источник соврешения/приема платежа не указан и номер отсутствует
    if temp[-1].isalpha():
        return row
    # если представлены сведения о банковской карте,
    # извлекаем номер банковской карты и
    # обрабатываем его функцией mask_bank_card
    elif temp[-1].isdigit() and len(temp[-1]) == 16:
        return ' '.join(temp[:-1]) + ' ' + mask_bank_card(temp[-1])
    # если представлены сведения о счете,
    # извлекаем номер счета и
    # обрабатываем его функцией mask_bank_account
    elif temp[-1].isdigit() and len(temp[-1]) == 20:
        return ' '.join(temp[:-1]) + ' ' + mask_bank_account(temp[-1])

In [ ]:
# на возможных значениях признака from проверим,
# что функция process_number отрабатывает корректно
t = process_number('Источник не указан')
print(f'Источник не указан: {t}')
t = process_number('Visa Classic 2842878893689012')
print(f'Банковская карта: {t}')
t = process_number('Счет 38611439522855669794')
print(f'Счет: {t}')

Источник не указан: Источник не указан
Банковская карта: Visa Classic 2842 87** **** 9012
Счет: Счет **9794


In [ ]:
# применим функцию process_number к признаку from
df['correct_from'] = df['from'].apply(lambda x: process_number(x))

In [ ]:
# проверим результат применения функции process_number
# к признаку from
df

,operation_amount,from,to,result,correct_from
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Источник не указан,Счет 90424923579946435907,08.12.2019 Открытие вклада,Источник не указан
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Visa Classic 2842878893689012,Счет 35158586384610753655,07.12.2019 Перевод организации,Visa Classic 2842 87** **** 9012
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Maestro 7810846596785568,Счет 43241152692663622869,19.11.2019 Перевод организации,Maestro 7810 84** **** 5568
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Счет 38611439522855669794,Счет 46765464282437878125,13.11.2019 Перевод со счета на счет,Счет **9794
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Источник не указан,Счет 77613226829885488381,05.11.2019 Открытие вклада,Источник не указан


In [ ]:
# к признаку result добавим сведения из признака correct_from;
# сразу добавим символ ->
df['result'] = df['result'] + ' ' + df['correct_from'] + ' ->'

In [ ]:
# удалим признаки from и correct_from, которые более не требуются
df = df.drop(['from', 'correct_from'], axis = 1)

In [ ]:
# проверим изменения в таблице
df

,operation_amount,to,result
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Счет 90424923579946435907,08.12.2019 Открытие вклада Источник не указан ->
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Счет 35158586384610753655,07.12.2019 Перевод организации Visa Classic 28...
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Счет 43241152692663622869,19.11.2019 Перевод организации Maestro 7810 84...
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Счет 46765464282437878125,13.11.2019 Перевод со счета на счет Счет **979...
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Счет 77613226829885488381,05.11.2019 Открытие вклада Источник не указан ->


Для обработки признака **to** также можно использовать функцию **process_number**

In [ ]:
# применим функцию process_number к признаку to
df['correct_to'] = df['to'].apply(lambda x: process_number(x))

In [ ]:
# проверим результат применения функции process_number
# к признаку to
df

,operation_amount,to,result,correct_to
11,"{'amount': '41096.24', 'currency': {'name': 'U...",Счет 90424923579946435907,08.12.2019 Открытие вклада Источник не указан ->,Счет **5907
70,"{'amount': '48150.39', 'currency': {'name': 'U...",Счет 35158586384610753655,07.12.2019 Перевод организации Visa Classic 28...,Счет **3655
40,"{'amount': '30153.72', 'currency': {'name': 'р...",Счет 43241152692663622869,19.11.2019 Перевод организации Maestro 7810 84...,Счет **2869
72,"{'amount': '62814.53', 'currency': {'name': 'р...",Счет 46765464282437878125,13.11.2019 Перевод со счета на счет Счет **979...,Счет **8125
38,"{'amount': '21344.35', 'currency': {'name': 'р...",Счет 77613226829885488381,05.11.2019 Открытие вклада Источник не указан ->,Счет **8381


In [ ]:
# к признаку result добавим сведения из признака correct_to
df['result'] = df['result'] + ' ' + df['correct_to']

In [ ]:
# удалим признаки to и correct_to, которые далее не требуются
df = df.drop(['to', 'correct_to'], axis = 1)

In [ ]:
# проверим произведенные изменения в таблице
df

,operation_amount,result
11,"{'amount': '41096.24', 'currency': {'name': 'U...",08.12.2019 Открытие вклада Источник не указан ...
70,"{'amount': '48150.39', 'currency': {'name': 'U...",07.12.2019 Перевод организации Visa Classic 28...
40,"{'amount': '30153.72', 'currency': {'name': 'р...",19.11.2019 Перевод организации Maestro 7810 84...
72,"{'amount': '62814.53', 'currency': {'name': 'р...",13.11.2019 Перевод со счета на счет Счет **979...
38,"{'amount': '21344.35', 'currency': {'name': 'р...",05.11.2019 Открытие вклада Источник не указан ...


In [ ]:
# выведем для контроля содержимое одной из ячеек result
df.iloc[2]['result']

'19.11.2019 Перевод организации Maestro 7810 84** **** 5568 -> Счет **2869'

Выведем содержимое некоторых ячеек признака **'operation_amount'**.

In [ ]:
df.iloc[2]['operation_amount']

{'amount': '30153.72', 'currency': {'name': 'руб.', 'code': 'RUB'}}

In [ ]:
df.iloc[1]['operation_amount']

{'amount': '48150.39', 'currency': {'name': 'USD', 'code': 'USD'}}

Необходимо извлечь из словаря сумму перевода и валюту перевода.

In [ ]:
# сформируем функцию для извлечения из соответствующей записи
# признака operation_amount суммы перевода и валюты перевода
def extract_amount_and_currency(
        row: dict,
        amount: str,
        currency: str,
        name: str) -> str:
    '''
    Функция извлекает по заданным ключам из полученного словаря
    сведения о сумме перевода и валюте перевода

    Params:
    :row: словарь со сведениями о сумме и валюте перевода
    :amount: ключ для получения суммы перевода
    :currency: первый ключ для получения валюты перевода
    :name: второй ключ для получения валюты перевода

    Return:
    :row: строка с указанием суммы перевода и валюты перевода

    Example:
    >>>new_row = extract_amount_and_currency(
        row,
        amount,
        currency,
        name)
    '''
    t1 = row.get(amount)
    t2 = row.get(currency).get(name)
    return t1 + ' ' + t2

In [ ]:
# проверим корректность отработки функции extract_amount_and_currency
t = df.iloc[2]['operation_amount']
print(extract_amount_and_currency(t, 'amount', 'currency', 'name'))

30153.72 руб.


In [ ]:
# применим функцию extract_amount_and_currency к признаку operation_amount
df['correct_operation_amount'] = df['operation_amount'].apply(
    lambda x:
    extract_amount_and_currency(x, 'amount', 'currency', 'name'))

In [ ]:
# добавим к сведениям об операциях клиентов информацию
# о сумме перевода и о валюте перевода
df['result'] = df['result'] + ' ' + df['correct_operation_amount']

In [ ]:
# удалим признаки 'operation_amount' и 'correct_operation_amount',
# которые далее не требуются
df = df.drop(['operation_amount', 'correct_operation_amount'], axis = 1)

In [ ]:
# сбросим индексы
df = df.reset_index(drop = True)

In [ ]:
# проверим произведенные изменения
df

,result
0,08.12.2019 Открытие вклада Источник не указан ...
1,07.12.2019 Перевод организации Visa Classic 28...
2,19.11.2019 Перевод организации Maestro 7810 84...
3,13.11.2019 Перевод со счета на счет Счет **979...
4,05.11.2019 Открытие вклада Источник не указан ...


Часть текста с информацией о пользователях скрыта. Настроим режим отображения.

In [ ]:
# увеличим число отображаемых символов в ячейке до 100
pd.options.display.max_colwidth = 100

In [ ]:
# проверим произведенные изменения
df

,result
0,08.12.2019 Открытие вклада Источник не указан -> Счет **5907 41096.24 USD
1,07.12.2019 Перевод организации Visa Classic 2842 87** **** 9012 -> Счет **3655 48150.39 USD
2,19.11.2019 Перевод организации Maestro 7810 84** **** 5568 -> Счет **2869 30153.72 руб.
3,13.11.2019 Перевод со счета на счет Счет **9794 -> Счет **8125 62814.53 руб.
4,05.11.2019 Открытие вклада Источник не указан -> Счет **8381 21344.35 руб.


In [ ]:
# сохраним результаты в .csv-файл
df.to_csv('client_5_last_operations.csv', index = False)

## <font color='mediumblue'>**ВЫВОДЫ**</font>

---

* В соответствии с техническим заданием подготовлены сведения о пяти последних совершенных (выполненных) операциях клиента в заданном формате.

* Результаты сохранены в файл **client_5_last_operations.csv**, который является неотъемлемой частью настоящего отчета.

* Последние пять операций совершены клиентом в период с **05.11.2019** по **08.12.2019**.

* Клиент дважды **открыл вклад** (в долларах США на сумму **41096.46** и в российских рублях на сумму **21344.35**)

* Клиент совершил два **перевода организации** (в долларах США на сумму **48150.39** и в российских рублях на сумму **30153.72**)

* Клиент совершил один **перевод со счета на счет** (в российских рублях на сумму **62814.53**)
